In [185]:
import os
import pandas as pd
import nltk
import requests
import string
from gensim.models import Word2Vec, KeyedVectors
import shutil
import numpy as np
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout, Attention, Concatenate
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
import shap


In [186]:
FINE_TUNED_DIR = '../../pretained_or_finetune-models'
REVIEWS_DATASET_DIR = '../../dataset'
UTILS_DIR = '../../utils'
NLTK_DATA_PATH = f"{FINE_TUNED_DIR}/nltk_data"

nltk.data.path.append(NLTK_DATA_PATH)

In [153]:
tourism_reviews_en = pd.read_csv(f"{REVIEWS_DATASET_DIR}/eng_reviews_emotion_classify.csv", encoding='utf-8')

columns_to_use = ['helpful_votes', 'location_id', 'review_id', 'review',
                  'review_subject', 'trip_type', 'rating',
                  'location_name', 'province', 'place_id', 'emotion', 'cleaned_review']
columns_to_train = ['location_id', 'review', 'rating']
review_df = tourism_reviews_en[columns_to_train]

In [154]:
review_df.describe()

,location_id,rating
count,5.909000e+03,5909.000000
mean,6.483167e+06,4.153833
std,4.877561e+06,1.067077
min,3.110380e+05,1.000000
25%,2.578107e+06,4.000000
50%,6.017599e+06,4.000000
75%,8.818088e+06,5.000000
max,2.714283e+07,5.000000


In [10]:
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5909 entries, 0 to 5908
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   location_id  5909 non-null   int64 
 1   review       5909 non-null   object
 2   rating       5909 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 138.6+ KB


In [155]:
review_df

,location_id,review,rating
0,2209612,Besides elegant grand palace and wat pra kaew ...,5
1,2209612,This is one of the last big king temples of Ba...,4
2,2209612,"It is beautiful, clean and free to enter. Both...",4
3,2209612,Wat Ratchabophit is a Buddhist temple located ...,5
4,2209612,Wat Ratchabophit is a historic royal temple bu...,5
...,...,...,...
5904,1831068,Is quite far from Korat Town or main highway. ...,3
5905,1831068,Prasat Pueai Noi is claimed to be the biggest ...,3
5906,1831068,I think this place is claimed to be the oldest...,4
5907,1831068,The compound comprises three brick buildings o...,4


In [15]:
binary_url = 'https://github.com/BrownDSI/word2vec-slim/releases/' + \
             'download/v0.1/GoogleNews-vectors-negative300-slim.bin.gz'
r = requests.get(binary_url)
open('google-word2vec-slim.bin.gz', 'wb').write(r.content);

#Load model contrained abot 300k of vocabs
model = KeyedVectors.load_word2vec_format('google-word2vec-slim.bin.gz', binary = True)

In [187]:
print(os.path.exists(f"{NLTK_DATA_PATH}/tokenizers/punkt")) 

True


In [188]:
# 1. Data Preparation
nltk.download('punkt', download_dir=NLTK_DATA_PATH)
stopwords = nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to ../../pretained_or_finetune-
[nltk_data]     models/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ittichaiboonyarakthunya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ittichaiboonyarakthunya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [189]:
from nltk.corpus import stopwords

custom_stopwords = set(stopwords.words('english'))

In [190]:
print(nltk.data.find('tokenizers/punkt'))

/Users/ittichaiboonyarakthunya/nltk_data/tokenizers/punkt


In [191]:
from nltk.stem import WordNetLemmatizer

stop_words = custom_stopwords
lemmatizer = WordNetLemmatizer()

In [192]:
text = "This"
text = text.lower()
text = re.sub(r'\d+', '', text)
text = re.sub(r'[^a-z\s]', '', text)
text = re.sub(r'\b(u|ur|b4)\b', 'you', text) 
tokens = nltk.word_tokenize(text) 
print(tokens)

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/Users/ittichaiboonyarakthunya/nltk_data'
    - '/Users/ittichaiboonyarakthunya/Library/Caches/pypoetry/virtualenvs/is-project-thailand-tourism-recommendation-qwL-LsgH-py3.12/nltk_data'
    - '/Users/ittichaiboonyarakthunya/Library/Caches/pypoetry/virtualenvs/is-project-thailand-tourism-recommendation-qwL-LsgH-py3.12/share/nltk_data'
    - '/Users/ittichaiboonyarakthunya/Library/Caches/pypoetry/virtualenvs/is-project-thailand-tourism-recommendation-qwL-LsgH-py3.12/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
   

In [28]:
def advanced_clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'[^a-z\s]', '', text)  # Remove special characters
    text = re.sub(r'\b(u|ur|b4)\b', 'you', text)  # Replace common abbreviations
    # return text
    words = word_tokenize(text)
    return words
    # words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    # words = [word for word in words if word not in string.punctuation]
    # return ' '.join(words)

In [29]:
# Apply preprocessing to the review texts
review_df['cleaned_review'] = review_df['review'].apply(advanced_clean_text)

# 2. สร้าง Word Embeddings ด้วย Word2Vec
# word2vec_model = Word2Vec(sentences=data['cleaned_review'], vector_size=300, window=5, min_count=1, sg=1)  # ใช้ Skip-Gram


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/Users/ittichaiboonyarakthunya/nltk_data'
    - '/Users/ittichaiboonyarakthunya/Library/Caches/pypoetry/virtualenvs/is-project-thailand-tourism-recommendation-qwL-LsgH-py3.12/nltk_data'
    - '/Users/ittichaiboonyarakthunya/Library/Caches/pypoetry/virtualenvs/is-project-thailand-tourism-recommendation-qwL-LsgH-py3.12/share/nltk_data'
    - '/Users/ittichaiboonyarakthunya/Library/Caches/pypoetry/virtualenvs/is-project-thailand-tourism-recommendation-qwL-LsgH-py3.12/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
***

In [27]:
review_df

,location_id,review,rating,cleaned_review
0,2209612,Besides elegant grand palace and wat pra kaew ...,5,besides elegant grand palace and wat pra kaew ...
1,2209612,This is one of the last big king temples of Ba...,4,this is one of the last big king temples of ba...
2,2209612,"It is beautiful, clean and free to enter. Both...",4,it is beautiful clean and free to enter both e...
3,2209612,Wat Ratchabophit is a Buddhist temple located ...,5,wat ratchabophit is a buddhist temple located ...
4,2209612,Wat Ratchabophit is a historic royal temple bu...,5,wat ratchabophit is a historic royal temple bu...
...,...,...,...,...
5904,1831068,Is quite far from Korat Town or main highway. ...,3,is quite far from korat town or main highway b...
5905,1831068,Prasat Pueai Noi is claimed to be the biggest ...,3,prasat pueai noi is claimed to be the biggest ...
5906,1831068,I think this place is claimed to be the oldest...,4,i think this place is claimed to be the oldest...
5907,1831068,The compound comprises three brick buildings o...,4,the compound comprises three brick buildings o...


In [ ]:
def text_to_word2vec(tokens, model, max_length=100):
    vector = np.zeros((max_length, 300))
    for i, word in enumerate(tokens):
        if i < max_length:
            if word in model.wv:
                vector[i] = model.wv[word]
    return vector

data['embedding'] = data['cleaned_review'].apply(lambda x: text_to_word2vec(x, word2vec_model))


In [ ]:
# 3. ทำ Sentence Padding หากใช้โมเดลเชิงลำดับ
max_length = 100
padded_sequences = pad_sequences(data['embedding'].tolist(), maxlen=max_length, padding='post', dtype='float32')

# ดูตัวอย่างข้อมูลที่ผ่านการแปลงแล้ว
print(padded_sequences[:5])